We initialize Python imports and opens a DuckDB connection that every later cell reuses

In [12]:
import warnings
from pathlib import Path
import duckdb
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

try:
    import statsmodels.genmod.generalized_linear_model as glm
    glm.SET_USE_BIC_LLF(True)
except Exception:
    pass

warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message="The bic value is computed using the deviance formula.*",
)

CWD = Path().resolve()

REPO_ROOT = None
DB_FILE = None

for p in [CWD] + list(CWD.parents):
    cand = p / "db" / "nflpa.duckdb"
    if cand.exists():
        REPO_ROOT = p
        DB_FILE = cand
        break

if DB_FILE is None:
    for p in [CWD] + list(CWD.parents):
        cand = p / "nflpa.duckdb"
        if cand.exists():
            REPO_ROOT = p
            DB_FILE = cand
            break

if DB_FILE is None:
    raise RuntimeError("Could not find nflpa.duckdb, expected db/nflpa.duckdb under the repo root")

try:
    if "con" in globals():
        try:
            con.close()
        except Exception:
            pass
except Exception:
    pass

try:
    con = duckdb.connect(str(DB_FILE))
except Exception as e:
    msg = str(e)
    if "Could not set lock on file" in msg or "Conflicting lock is held" in msg:
        raise RuntimeError(
            "DuckDB is locked by another python process. Close other notebook kernels using nflpa.duckdb, or kill the PID shown in the error, then rerun."
        ) from e
    raise

SEASON_COL = "season"
WEEK_COL = "week"
TEAM_COL = "team"

MODEL_VIEW = "team_week_panel_nextweek_model"

exists_df = con.execute(f"""
SELECT
  COUNT(*) AS n
FROM information_schema.tables
WHERE table_schema = 'main'
  AND table_name = '{MODEL_VIEW}'
  AND table_type IN ('BASE TABLE', 'VIEW')
""").df()

if int(exists_df["n"].iloc[0]) == 0:
    raise RuntimeError(f"Missing {MODEL_VIEW}, run notebook 11 to create the model view")

print("connected db", str(DB_FILE))
print("model view", MODEL_VIEW)

connected db /Users/ramko/Desktop/2025-26-NFLPA-Data-Analytics-Case-Competition/db/nflpa.duckdb
model view team_week_panel_nextweek_model


Quick sanity check to confirm that 'has_next_week' is always 1 in the model view and that the view has unique season week team keys

In [13]:
con.execute(f"""
SELECT
  COUNT(*) AS rows_model,
  SUM(CASE WHEN has_next_week != 1 THEN 1 ELSE 0 END) AS bad_has_next_week,
  SUM(CASE WHEN Inj_Off_Next_w IS NULL THEN 1 ELSE 0 END) AS null_outcome_off
FROM {MODEL_VIEW}
""").df()
con.execute(f"""
SELECT
  COUNT(*) AS dup_rows
FROM (
  SELECT
    season,
    week,
    team AS team_key,
    COUNT(*) AS n
  FROM {MODEL_VIEW}
  GROUP BY 1,2,3
  HAVING COUNT(*) > 1
) d
""").df()

check_has_next = con.execute(f"""
SELECT
  COUNT(*) AS rows_model,
  SUM(CASE WHEN has_next_week != 1 THEN 1 ELSE 0 END) AS bad_has_next_week,
  SUM(CASE WHEN Inj_Off_Next_w IS NULL THEN 1 ELSE 0 END) AS null_outcome_off
FROM {MODEL_VIEW}
""").df()

check_dups = con.execute(f"""
SELECT
  COUNT(*) - COUNT(DISTINCT season || '-' || week || '-' || team) AS n_dup_keys
FROM {MODEL_VIEW}
""").df()

print(check_has_next.to_string(index=False))
print()
print(check_dups.to_string(index=False))

 rows_model  bad_has_next_week  null_outcome_off
       5950                0.0               0.0

 n_dup_keys
          0


Quick sanity check to confirm that all required columns exist. We normalize any naming differences so formulas are stable

In [4]:
cols = con.execute(f"DESCRIBE {MODEL_VIEW}").df()["column_name"].tolist()
cols_set = set(cols)

def pick_first_present(options):
    for o in options:
        if o in cols_set:
            return o
    return None

TEAM_COL = "team" if "team" in cols_set else ("team_key" if "team_key" in cols_set else None)
if TEAM_COL is None:
    raise RuntimeError("Missing team column, expected team or team_key in model view")

POINTS_FOR_COL = pick_first_present(["points_for_w", "points_for"])
POINTS_AGAINST_COL = pick_first_present(["points_against_w", "points_against"])
SCORE_DIFF_COL = pick_first_present(["score_diff_w", "score_diff"])
OFF_YPP_COL = pick_first_present(["off_yards_per_play_w", "Off_yards_per_play_w"])
CWI_COL = pick_first_present(["Cumulative_Workload_Index_w", "cumulative_workload_index_w"])

if POINTS_FOR_COL is None or POINTS_AGAINST_COL is None:
    raise RuntimeError("Missing points columns, expected points_for and points_against variants")

if SCORE_DIFF_COL is None:
    raise RuntimeError("Missing score diff column, expected score_diff_w")

if OFF_YPP_COL is None:
    raise RuntimeError("Missing offensive yards per play column, expected off_yards_per_play_w")

if CWI_COL is None:
    raise RuntimeError("Missing workload index column, expected Cumulative_Workload_Index_w")

OUTCOME_OFF = "Inj_Off_Next_w"
LAG_COLS = [
    "ST_Shock_NonScore_w_minus_1",
    "ST_Shock_NonScore_w_minus_2",
    "ST_Shock_NonScore_w_minus_3",
]

SHOCK_COL_MAIN = "ST_Shock_NonScore_Roll_w" if "ST_Shock_NonScore_Roll_w" in cols_set else "ST_Shock_NonScore_w"

required = [
    SEASON_COL, WEEK_COL, TEAM_COL,
    OUTCOME_OFF,
    "Inj_Off_Last_w",
    "blowout_flag_w",
    "short_week_flag_w",
    "bye_last_week_flag_w",
    "home_flag_w",
    "offensive_snaps_w",
    "defensive_snaps_w",
    SHOCK_COL_MAIN,
    "ST_Vol_NonScore_w",
    "Cum_Shocks_NonScore_w",
    OFF_YPP_COL,
    SCORE_DIFF_COL,
    POINTS_FOR_COL,
    POINTS_AGAINST_COL,
    CWI_COL,
] + LAG_COLS

missing = [c for c in required if c not in cols_set]
if missing:
    raise RuntimeError("Missing required Step 15 columns in model view, " + ", ".join(missing))

print("team column", TEAM_COL)
print("shock column main", SHOCK_COL_MAIN)
print("points_for column", POINTS_FOR_COL)
print("points_against column", POINTS_AGAINST_COL)
print("score_diff column", SCORE_DIFF_COL)
print("off_ypp column", OFF_YPP_COL)
print("workload index column", CWI_COL)

team column team
shock column main ST_Shock_NonScore_w
points_for column points_for
points_against column points_against
score_diff column score_diff_w
off_ypp column off_yards_per_play_w
workload index column Cumulative_Workload_Index_w


We build the modeling frame from the model view, then construct shock variables and interactions, and finally drop rows only where truly required inputs are missing

In [5]:
select_cols = [
    SEASON_COL,
    WEEK_COL,
    TEAM_COL,
    OUTCOME_OFF,
    "Inj_Off_Last_w",
    "blowout_flag_w",
    "short_week_flag_w",
    "bye_last_week_flag_w",
    "home_flag_w",
    "offensive_snaps_w",
    "defensive_snaps_w",
    SHOCK_COL_MAIN,
    "ST_Vol_NonScore_w",
    "Cum_Shocks_NonScore_w",
] + LAG_COLS + [
    POINTS_FOR_COL,
    POINTS_AGAINST_COL,
    SCORE_DIFF_COL,
    OFF_YPP_COL,
    CWI_COL,
]

df = con.execute(f"SELECT {', '.join(select_cols)} FROM {MODEL_VIEW}").df()

rename_map = {}
if POINTS_FOR_COL != "points_for":
    rename_map[POINTS_FOR_COL] = "points_for"
if POINTS_AGAINST_COL != "points_against":
    rename_map[POINTS_AGAINST_COL] = "points_against"
if SCORE_DIFF_COL != "score_diff_w":
    rename_map[SCORE_DIFF_COL] = "score_diff_w"
if OFF_YPP_COL != "off_yards_per_play_w":
    rename_map[OFF_YPP_COL] = "off_yards_per_play_w"
if CWI_COL != "Cumulative_Workload_Index_w":
    rename_map[CWI_COL] = "Cumulative_Workload_Index_w"

df = df.rename(columns=rename_map)

df[TEAM_COL] = df[TEAM_COL].astype(str)
df[SEASON_COL] = df[SEASON_COL].astype(int)
df[WEEK_COL] = df[WEEK_COL].astype(int)

df["season_week"] = (df[SEASON_COL] * 100 + df[WEEK_COL]).astype(int)

df["shock_nonscore"] = df[SHOCK_COL_MAIN].fillna(0).astype(int)
df["blowout_flag_w"] = df["blowout_flag_w"].fillna(0).astype(int)
df["shock_x_blowout"] = (df["shock_nonscore"] * df["blowout_flag_w"]).astype(int)

for c in LAG_COLS:
    df[c] = df[c].fillna(0).astype(int)

flag_cols = ["short_week_flag_w", "bye_last_week_flag_w", "home_flag_w"]
for c in flag_cols:
    df[c] = df[c].fillna(0).astype(int)

df["ST_Vol_NonScore_w"] = df["ST_Vol_NonScore_w"].fillna(0).astype(float)
df["Cum_Shocks_NonScore_w"] = df["Cum_Shocks_NonScore_w"].fillna(0).astype(float)

must_not_be_null = [
    OUTCOME_OFF,
    "Inj_Off_Last_w",
    "offensive_snaps_w",
    "defensive_snaps_w",
    "points_for",
    "points_against",
    "score_diff_w",
    "off_yards_per_play_w",
    "Cumulative_Workload_Index_w",
]

before = len(df)
df = df.dropna(subset=must_not_be_null).reset_index(drop=True)
after = len(df)

print("rows before dropna", before)
print("rows after dropna", after)
df.head(3)

rows before dropna 5950
rows after dropna 5950


,season,week,team,Inj_Off_Next_w,Inj_Off_Last_w,blowout_flag_w,short_week_flag_w,bye_last_week_flag_w,home_flag_w,offensive_snaps_w,...,ST_Shock_NonScore_w_minus_2,ST_Shock_NonScore_w_minus_3,points_for,points_against,score_diff_w,off_yards_per_play_w,Cumulative_Workload_Index_w,season_week,shock_nonscore,shock_x_blowout
0,2012,1,ATL,2.0,0.0,1,0,0,0,55.0,...,0,0,40,24,16,6.836364,-3.940011,201201,0,0
1,2012,2,ATL,2.0,2.0,0,0,0,1,65.0,...,0,0,27,21,6,4.412698,-3.638251,201202,0,0
2,2012,3,ATL,2.0,2.0,1,0,0,0,69.0,...,0,0,27,3,24,5.565217,-2.938346,201203,0,0


Quick sanity check to confirm that the modeling frame has no nulls in key predictors and that core numeric fields look finite and usable

In [6]:
check_cols = [
    OUTCOME_OFF,
    "shock_nonscore",
    "ST_Vol_NonScore_w",
    "Cum_Shocks_NonScore_w",
    "points_for",
    "points_against",
    "score_diff_w",
    "offensive_snaps_w",
    "defensive_snaps_w",
    "off_yards_per_play_w",
    "Inj_Off_Last_w",
    "Cumulative_Workload_Index_w",
]

null_counts = {c: int(df[c].isna().sum()) for c in check_cols}
nonfinite = {
    "ST_Vol_NonScore_w_nonfinite": int((~np.isfinite(df["ST_Vol_NonScore_w"].astype(float))).sum()),
    "Cum_Shocks_NonScore_w_nonfinite": int((~np.isfinite(df["Cum_Shocks_NonScore_w"].astype(float))).sum()),
    "off_yards_per_play_w_nonfinite": int((~np.isfinite(df["off_yards_per_play_w"].astype(float))).sum()),
    "CWI_nonfinite": int((~np.isfinite(df["Cumulative_Workload_Index_w"].astype(float))).sum()),
}

print("null counts", null_counts)
print("nonfinite counts", nonfinite)
print("n rows", len(df))
print("n teams", df[TEAM_COL].nunique())

null counts {'Inj_Off_Next_w': 0, 'shock_nonscore': 0, 'ST_Vol_NonScore_w': 0, 'Cum_Shocks_NonScore_w': 0, 'points_for': 0, 'points_against': 0, 'score_diff_w': 0, 'offensive_snaps_w': 0, 'defensive_snaps_w': 0, 'off_yards_per_play_w': 0, 'Inj_Off_Last_w': 0, 'Cumulative_Workload_Index_w': 0}
nonfinite counts {'ST_Vol_NonScore_w_nonfinite': 0, 'Cum_Shocks_NonScore_w_nonfinite': 0, 'off_yards_per_play_w_nonfinite': 0, 'CWI_nonfinite': 0}
n rows 5950
n teams 35


We compute dispersion diagnostics for the offensive next week injury count so the Poisson baseline and Negative Binomial contingency choice is explicitly grounded in the data

In [14]:
def outcome_dispersion_stats(y: pd.Series) -> dict:
    y = y.astype(float)
    mean = float(y.mean())
    var = float(y.var(ddof=1))
    share_zero = float((y == 0).mean())
    return {
        "mean": mean,
        "var": var,
        "var_over_mean": (var / mean) if mean > 0 else np.nan,
        "share_zero": share_zero,
        "max": float(y.max()),
    }

stats_off = outcome_dispersion_stats(df[OUTCOME_OFF])

print("offense outcome", OUTCOME_OFF)
for k in ["mean", "var", "var_over_mean", "share_zero", "max"]:
    print(k, stats_off[k])

material_overdispersion_off = (not np.isnan(stats_off["var_over_mean"])) and (stats_off["var_over_mean"] >= 1.5)
print("material overdispersion offense", bool(material_overdispersion_off))

offense outcome Inj_Off_Next_w
mean 1.9201680672268908
var 2.1612169830110557
var_over_mean 1.125535321568121
share_zero 0.17714285714285713
max 9.0
material overdispersion offense False


We fit Model B using Poisson with team clustered standard errors and also fit Negative Binomial on the same formulas as a contingency and direct comparison

In [9]:
FE_TEAM = f"C({TEAM_COL})"
FE_TIME = "C(season_week)"
cluster_groups = df[TEAM_COL]

def fit_count_model_poisson_glm(formula: str, data: pd.DataFrame, groups: pd.Series):
    m = smf.glm(formula=formula, data=data, family=sm.families.Poisson())
    r = m.fit(cov_type="cluster", cov_kwds={"groups": groups})
    return r

def fit_count_model_negative_binomial_mle(formula: str, data: pd.DataFrame, groups: pd.Series, maxiter: int = 200):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        m = smf.negativebinomial(formula=formula, data=data)
        r = m.fit(disp=False, maxiter=maxiter, cov_type="cluster", cov_kwds={"groups": groups})
    return r

def _safe_float(x):
    try:
        return float(x)
    except Exception:
        return np.nan

def print_nb_compare(pois_res, nb_res, name: str):
    if nb_res is None:
        print(name, "negative binomial fit failed")
        return
    aic_pois = _safe_float(getattr(pois_res, "aic", np.nan))
    aic_nb = _safe_float(getattr(nb_res, "aic", np.nan))
    alpha = np.nan
    try:
        if hasattr(nb_res, "params") and ("alpha" in nb_res.params.index):
            alpha = _safe_float(nb_res.params["alpha"])
    except Exception:
        pass
    print(name, "aic_poisson", aic_pois, "aic_nb", aic_nb, "alpha", alpha)

exposure_terms = [
    "shock_nonscore",
    "shock_x_blowout",
    "ST_Vol_NonScore_w",
    "Cum_Shocks_NonScore_w",
    "ST_Shock_NonScore_w_minus_1",
    "ST_Shock_NonScore_w_minus_2",
    "ST_Shock_NonScore_w_minus_3",
]

control_terms_base = [
    "offensive_snaps_w",
    "defensive_snaps_w",
    "blowout_flag_w",
    "short_week_flag_w",
    "bye_last_week_flag_w",
    "home_flag_w",
    "off_yards_per_play_w",
    "Inj_Off_Last_w",
    "Cumulative_Workload_Index_w",
]

script_specs = [
    ("points_for_diff", ["points_for", "score_diff_w"]),
    ("points_against_diff", ["points_against", "score_diff_w"]),
    ("points_for_against", ["points_for", "points_against"]),
]

def build_formula(outcome: str, script_terms: list) -> str:
    rhs = exposure_terms + control_terms_base + script_terms + [FE_TEAM, FE_TIME]
    return outcome + " ~ " + " + ".join(rhs)

fits = []

for tag, script_terms in script_specs:
    f = build_formula(OUTCOME_OFF, script_terms)

    try:
        pois = fit_count_model_poisson_glm(f, df, cluster_groups)
    except Exception as e:
        print("poisson failed", tag, str(e))
        continue

    nb = None
    try:
        nb = fit_count_model_negative_binomial_mle(f, df, cluster_groups)
    except Exception as e:
        print("negative binomial failed", tag, str(e))

    fits.append((tag, f, pois, nb))
    print("fit ok", tag)
    print_nb_compare(pois, nb, "nb compare " + tag)
    print()

if len(fits) == 0:
    raise RuntimeError("No Model B specifications fit successfully")

preferred_order = ["points_for_diff", "points_against_diff", "points_for_against"]
fits_sorted = sorted(
    fits,
    key=lambda x: preferred_order.index(x[0]) if x[0] in preferred_order else 999
)

spec_tag_off, formula_off_used, pois_off, nb_off = fits_sorted[0]

print("selected Model B spec", spec_tag_off)
print(formula_off_used)

fit ok points_for_diff
nb compare points_for_diff aic_poisson 20219.032919018013 aic_nb 20221.054362502076 alpha 9.047476014217315e-05

fit ok points_against_diff
nb compare points_against_diff aic_poisson 20219.032919018013 aic_nb 20221.034594437268 alpha 6.410932196775635e-06

fit ok points_for_against
nb compare points_for_against aic_poisson 20219.032919018013 aic_nb 20221.033822834066 alpha 6.093836797495602e-06

selected Model B spec points_for_diff
Inj_Off_Next_w ~ shock_nonscore + shock_x_blowout + ST_Vol_NonScore_w + Cum_Shocks_NonScore_w + ST_Shock_NonScore_w_minus_1 + ST_Shock_NonScore_w_minus_2 + ST_Shock_NonScore_w_minus_3 + offensive_snaps_w + defensive_snaps_w + blowout_flag_w + short_week_flag_w + bye_last_week_flag_w + home_flag_w + off_yards_per_play_w + Inj_Off_Last_w + Cumulative_Workload_Index_w + points_for + score_diff_w + C(team) + C(season_week)


We print and interpret the key exposure coefficients as IRR with clustered confidence intervals for the selected Model B specification

In [10]:
key_terms = ["shock_nonscore", "shock_x_blowout", "ST_Vol_NonScore_w", "Cum_Shocks_NonScore_w"]

def print_key_terms(res, name: str):
    print(name, "nobs", int(res.nobs))
    for t in key_terms:
        if t in res.params.index:
            beta = float(res.params[t])
            se = float(res.bse[t])
            p = float(res.pvalues[t])
            irr = float(np.exp(beta))
            ci_lo = float(np.exp(beta - 1.96 * se))
            ci_hi = float(np.exp(beta + 1.96 * se))
            print(t, "beta", beta, "se", se, "irr", irr, "ci", (ci_lo, ci_hi), "p", p)

print_key_terms(pois_off, "Model B poisson selected")
print()
if nb_off is not None:
    print_key_terms(nb_off, "Model B negative binomial selected")

Model B poisson selected nobs 5950
shock_nonscore beta 0.02645322493954999 se 0.039953004586588146 irr 1.0268062172155061 ci (0.9494668484820635, 1.1104453087519617) p 0.5079016726533399
shock_x_blowout beta 0.11599140431648264 se 0.06520516146316443 irr 1.1229862192576332 ci (0.9882587605331672, 1.2760808191188595) p 0.07526133218956722
ST_Vol_NonScore_w beta 0.019592017281999954 se 0.016189446136112992 irr 1.0197852004058034 ci (0.9879340868791653, 1.0526631976551113) p 0.2262128317914499
Cum_Shocks_NonScore_w beta -0.019337511166432314 se 0.019825641138828003 irr 0.9808482591304181 ci (0.9434652292455278, 1.0197125210523301) p 0.32937236808128845

Model B negative binomial selected nobs 5950
shock_nonscore beta 0.026540285735443814 se 0.039911199699507384 irr 1.0268956156734996 ci (0.9496273203955314, 1.1104509978190575) p 0.5060611031924241
shock_x_blowout beta 0.11576411856578803 se 0.06523207164973928 irr 1.1227310094956393 ci (0.9879820574570939, 1.275858109131541) p 0.075955638

We export Model B results to DuckDB and CSV in a tidy format, including the both Poisson and Negative Binomial rows when available

In [11]:
def tidy_res(res, model_name: str, outcome_name: str, spec_tag: str, irr_terms: list) -> pd.DataFrame:
    params = res.params.copy()
    bse = res.bse.copy()
    pvals = res.pvalues.copy()

    out = pd.DataFrame({
        "model": model_name,
        "spec_tag": spec_tag,
        "outcome": outcome_name,
        "term": params.index.astype(str),
        "beta": params.values.astype(float),
        "se_cluster": bse.values.astype(float),
        "pvalue": pvals.values.astype(float),
    })

    irr_terms_set = set(irr_terms)
    out["is_irr_term"] = out["term"].apply(lambda x: 1 if x in irr_terms_set else 0)

    out["irr"] = np.nan
    out["ci_low_irr"] = np.nan
    out["ci_high_irr"] = np.nan

    mask = out["is_irr_term"] == 1
    out.loc[mask, "irr"] = np.exp(out.loc[mask, "beta"])
    out.loc[mask, "ci_low_irr"] = np.exp(out.loc[mask, "beta"] - 1.96 * out.loc[mask, "se_cluster"])
    out.loc[mask, "ci_high_irr"] = np.exp(out.loc[mask, "beta"] + 1.96 * out.loc[mask, "se_cluster"])

    out["nobs"] = int(res.nobs)
    out["aic"] = float(getattr(res, "aic", np.nan))
    out["bic"] = float(getattr(res, "bic", np.nan))
    out["llf"] = float(getattr(res, "llf", np.nan))
    return out

results = []
results.append(tidy_res(pois_off, "poisson_modelB", OUTCOME_OFF, spec_tag_off, key_terms))
if nb_off is not None:
    results.append(tidy_res(nb_off, "negative_binomial_modelB", OUTCOME_OFF, spec_tag_off, key_terms))

results_df = pd.concat(results, ignore_index=True)
key_keep = set(key_terms)
results_df["is_key_term"] = results_df["term"].apply(lambda x: 1 if x in key_keep else 0)

con.register("step15_modelB_tmp", results_df)
con.execute("CREATE OR REPLACE TABLE step15_modelB_results AS SELECT * FROM step15_modelB_tmp")
con.unregister("step15_modelB_tmp")

out_dir = Path("../outputs")
out_dir.mkdir(parents=True, exist_ok=True)
csv_path = out_dir / "step15_modelB_results.csv"
results_df.to_csv(csv_path, index=False)

print("wrote duckdb table step15_modelB_results")
print("wrote csv", csv_path.resolve())

results_df.query("is_key_term == 1").sort_values(["model", "term"]).head(20)

wrote duckdb table step15_modelB_results
wrote csv /Users/ramko/Desktop/2025-26-NFLPA-Data-Analytics-Case-Competition/outputs/step15_modelB_results.csv


,model,spec_tag,outcome,term,beta,se_cluster,pvalue,is_irr_term,irr,ci_low_irr,ci_high_irr,nobs,aic,bic,llf,is_key_term
513,negative_binomial_modelB,points_for_diff,Inj_Off_Next_w,Cum_Shocks_NonScore_w,-0.019351,0.019866,0.330016,1,0.980835,0.943378,1.019779,5950,20221.054363,21994.208185,-9845.527181,1
512,negative_binomial_modelB,points_for_diff,Inj_Off_Next_w,ST_Vol_NonScore_w,0.019782,0.016166,0.221076,1,1.019979,0.988167,1.052816,5950,20221.054363,21994.208185,-9845.527181,1
510,negative_binomial_modelB,points_for_diff,Inj_Off_Next_w,shock_nonscore,0.026540,0.039911,0.506061,1,1.026896,0.949627,1.110451,5950,20221.054363,21994.208185,-9845.527181,1
511,negative_binomial_modelB,points_for_diff,Inj_Off_Next_w,shock_x_blowout,0.115764,0.065232,0.075956,1,1.122731,0.987982,1.275858,5950,20221.054363,21994.208185,-9845.527181,1
249,poisson_modelB,points_for_diff,Inj_Off_Next_w,Cum_Shocks_NonScore_w,-0.019338,0.019826,0.329372,1,0.980848,0.943465,1.019713,5950,20219.032919,21985.495595,-9845.516460,1
248,poisson_modelB,points_for_diff,Inj_Off_Next_w,ST_Vol_NonScore_w,0.019592,0.016189,0.226213,1,1.019785,0.987934,1.052663,5950,20219.032919,21985.495595,-9845.516460,1
246,poisson_modelB,points_for_diff,Inj_Off_Next_w,shock_nonscore,0.026453,0.039953,0.507902,1,1.026806,0.949467,1.110445,5950,20219.032919,21985.495595,-9845.516460,1
247,poisson_modelB,points_for_diff,Inj_Off_Next_w,shock_x_blowout,0.115991,0.065205,0.075261,1,1.122986,0.988259,1.276081,5950,20219.032919,21985.495595,-9845.516460,1
